<a href="https://colab.research.google.com/github/KiimKii/nsrd/blob/main/BERT_NAVER_SHOPPING_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

     |████████████████████████████████| 2.3MB 7.5MB/s 
     |████████████████████████████████| 901kB 47.1MB/s 
     |████████████████████████████████| 3.3MB 49.3MB/s 


In [2]:
% pip install sentencepiece

     |████████████████████████████████| 1.2MB 8.5MB/s 


In [3]:
% pip install kobert-transformers

In [4]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-tocm67me
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-tocm67me
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12718 sha256=f0182991cca12a691652b2d6f2d72e8d61ec21c0ef5b045fb2f57be6a888b23e
  Stored in directory: /tmp/pip-ephem-wheel-cache-_4fshsv1/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [5]:
import tensorflow as tf
import torch
from torch.nn import functional as F
from tqdm.notebook import tqdm

from transformers import BertModel, DistilBertModel
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [6]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from kobert.utils import get_tokenizer
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

### 깃허브 데이터셋 호출

In [7]:
# 네이버 쇼핑 리뷰 감정분석 데이터 다운로드
!git clone https://github.com/KiimKii/nsrd.git

Cloning into 'nsrd'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 43 (delta 16), reused 25 (delta 6), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [8]:
# 디렉토리의 파일 목록
!ls nsrd -la

total 42540
drwxr-xr-x 3 root root     4096 Jun 17 02:13  .
drwxr-xr-x 1 root root     4096 Jun 17 02:13  ..
-rw-r--r-- 1 root root   110339 Jun 17 02:13  처음
-rw-r--r-- 1 root root   155312 Jun 17 02:13 '아이폰 리뷰 워드클라우드1.ipynb'
-rw-r--r-- 1 root root   222832 Jun 17 02:13  BERT_NAVER_SHOPPING.ipynb
-rw-r--r-- 1 root root   161047 Jun 17 02:13  BERT_NAVER_SHOPPING_v2.ipynb
-rw-r--r-- 1 root root   240231 Jun 17 02:13  BERT_NAVER_SHOPPING_v3.ipynb
-rw-r--r-- 1 root root   126068 Jun 17 02:13  berttest.ipynb
-rw-r--r-- 1 root root     5154 Jun 17 02:13 'Data Crawling-Copy1.ipynb'
drwxr-xr-x 8 root root     4096 Jun 17 02:13  .git
-rw-r--r-- 1 root root    22818 Jun 17 02:13 '네이버 쇼핑 리뷰 데이터셋 정리.ipynb'
-rw-r--r-- 1 root root   165354 Jun 17 02:13 '워드클라우드_네이버 쇼핑 리뷰 데이터.ipynb'
-rw-r--r-- 1 root root    77883 Jun 17 02:13  애플로고.jpg
-rw-r--r-- 1 root root 21023837 Jun 17 02:13  naver_shop.txt
-rw-r--r-- 1 root root 21023837 Jun 17 02:13  nsrd.txt
-rw-r--r-- 1 root root   107608 Jun 17 02:13  워드클라우

In [9]:
main = pd.read_csv("nsrd/nsrd.txt", sep='\t')
main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   ratings  200000 non-null  int64 
 1   reviews  200000 non-null  object
 2   label    200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [10]:
main = main.reset_index()
main.head()

,index,ratings,reviews,label
0,0,5,배공빠르고 굿,1
1,1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


In [11]:
main2 = main[['index','reviews','label']]

main3 = main2.rename(columns={'index':'id'})
main3
main3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   id       200000 non-null  int64 
 1   reviews  200000 non-null  object
 2   label    200000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 4.6+ MB


In [12]:
main3.head()

,id,reviews,label
0,0,배공빠르고 굿,1
1,1,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고,0
2,2,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...,1
3,3,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...,0
4,4,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ,1


In [13]:
train = main3[:150000]
display(train.tail())

test = main3[150000:]
test.head()


,id,reviews,label
149995,149995,아주예뻐요 진짜예뻐요 욕실 기존거울철거하고 이거두개 나란히 달았는데 분위기가 사네요...,1
149996,149996,지퍼가 쭉 빠져버리네요 박음질이 정말 엉성한듯.. 고치기 어려워 버려요.. 좀 튼튼...,0
149997,149997,냄새는 좋은대 지속력이 생각보다 없는거 같아요,0
149998,149998,너무맛잇개 잘먹음요 신선해서 애들살아서오공 알배기가 많지는 안앗지만몇마리잇어서좋앗어...,1
149999,149999,껍질이 몇마리 깨진게 있긴 했지만 대체적으로 신선해서 좋았습니다.,1


,id,reviews,label
150000,150000,비싸게 구매했는데 귀가 너무 불편해요. 아파요ㅜㅜ,0
150001,150001,붙이는 부분이 번거럽고 부착했을때도 자연스럽지 않아서 오래 하기는 안예뻤어요. 그나...,0
150002,150002,배송도빠르고맛도괜찮은듯,1
150003,150003,정말 만족스럽습니다 빠른 배송도 감사합니다,1
150004,150004,아바이순대에 같이 나오는 명태무침맛에서 조금 아쉬웠어요!,1


In [14]:
print(train.shape)
print(test.shape)

(150000, 3)
(50000, 3)


#### **전처리 1**


In [15]:
# 리뷰 문장 추출
sentences = train['reviews']
sentences[:10]

0                                              배공빠르고 굿
1                        택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2    아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3    선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4                    민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
5    비추합니다 계란 뒤집을 때 완전 불편해요 ㅠㅠ 코팅도 묻어나고 보기엔 예쁘고 실용적...
6    주문을 11월6에 시켰는데 11월16일에 배송이 왔네요 ㅎㅎㅎ 여기 회사측과는 전화...
7                            넉넉한 길이로 주문했는데도 안 맞네요 별로예요
8                            보폴이 계속 때처럼 나오다가 지금은 안나네요~
9    110인데 전문속옷브랜드 위생팬티105보다 작은듯해요. 불편해요. 밴딩부분이 다 신...
Name: reviews, dtype: object

In [16]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 배공빠르고 굿 [SEP]',
 '[CLS] 택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고 [SEP]',
 '[CLS] 아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요. [SEP]',
 '[CLS] 선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전화했더니 바로주신다했지만 배송도 누락되어있었네요.. 확인안하고 바로 선물했으면 큰일날뻔했네요..이렇게 배송이 오래걸렸으면 사는거 다시 생각했을거같아요 아쉽네요.. [SEP]',
 '[CLS] 민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ [SEP]',
 '[CLS] 비추합니다 계란 뒤집을 때 완전 불편해요 ㅠㅠ 코팅도 묻어나고 보기엔 예쁘고 실용적으로 보였는데 생각보다 진짜 별로입니다. [SEP]',
 '[CLS] 주문을 11월6에 시켰는데 11월16일에 배송이 왔네요 ㅎㅎㅎ 여기 회사측과는 전화도 안되고 아무런 연락을 받을수가 없으니 답답하신 분들은 다른곳에서 사시는거 추천드립니다 [SEP]',
 '[CLS] 넉넉한 길이로 주문했는데도 안 맞네요 별로예요 [SEP]',
 '[CLS] 보폴이 계속 때처럼 나오다가 지금은 안나네요~ [SEP]',
 '[CLS] 110인데 전문속옷브랜드 위생팬티105보다 작은듯해요. 불편해요. 밴딩부분이 다 신축성없는 일반실로 되어있어 빅사이즈임에도 빅사이즈같지않아요. 입고벗을때 편하게 밴딩부분이 늘어나고 입었을때도 밴딩이 잡아주어야하는데 말이죠. [SEP]']

In [17]:
# 라벨 추출
labels = train['label'].values
labels

array([1, 0, 1, ..., 0, 1, 1])

In [18]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])


[CLS] 배공빠르고 굿 [SEP]
['[CLS]', '배', '##공', '##빠', '##르고', '굿', '[SEP]']


In [19]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9330,  28000, 119008,  81220,   8915,    102,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [20]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [21]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([  101,  9766, 70915, 22458, 23160,  9543, 10739,  9004, 32537, 14801,
        12965, 48549,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [22]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

### **전처리 2**

In [23]:
# 리뷰 문장 추출
sentences = test['reviews']
sentences[:10]

150000                          비싸게 구매했는데 귀가 너무 불편해요. 아파요ㅜㅜ
150001    붙이는 부분이 번거럽고 부착했을때도 자연스럽지 않아서 오래 하기는 안예뻤어요. 그나...
150002                                         배송도빠르고맛도괜찮은듯
150003                              정말 만족스럽습니다 빠른 배송도 감사합니다
150004                      아바이순대에 같이 나오는 명태무침맛에서 조금 아쉬웠어요!
150005    틀이 잡히는 느낌이 없어요, 모양을 유지하는 것이 약해서 지퍼 열었다 닫을 때 한손...
150006                    소재면 소재 신축성이면 신축성~!! 뭐하나 단점이 업네요~~
150007    2점을 준 이유는 양말이 발 부분이 너무 얇아요 발목 위로는 골지라 괜찮은데 그 아...
150008                                  배송 빠르고 원하는 상품구입했어요.
150009                         글루 나오는 입구를 뚫는방법도 없고..접착도 약하고
Name: reviews, dtype: object

In [24]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 비싸게 구매했는데 귀가 너무 불편해요. 아파요ㅜㅜ [SEP]',
 '[CLS] 붙이는 부분이 번거럽고 부착했을때도 자연스럽지 않아서 오래 하기는 안예뻤어요. 그나마 한개도 사용하지 않고 그냥 두고 있네요. [SEP]',
 '[CLS] 배송도빠르고맛도괜찮은듯 [SEP]',
 '[CLS] 정말 만족스럽습니다 빠른 배송도 감사합니다 [SEP]',
 '[CLS] 아바이순대에 같이 나오는 명태무침맛에서 조금 아쉬웠어요! [SEP]',
 '[CLS] 틀이 잡히는 느낌이 없어요, 모양을 유지하는 것이 약해서 지퍼 열었다 닫을 때 한손으로는 지퍼가 울어서 두손으로해야 해요 [SEP]',
 '[CLS] 소재면 소재 신축성이면 신축성~!! 뭐하나 단점이 업네요~~ [SEP]',
 '[CLS] 2점을 준 이유는 양말이 발 부분이 너무 얇아요 발목 위로는 골지라 괜찮은데 그 아래는 너무얇아요 [SEP]',
 '[CLS] 배송 빠르고 원하는 상품구입했어요. [SEP]',
 '[CLS] 글루 나오는 입구를 뚫는방법도 없고..접착도 약하고 [SEP]']

In [25]:
# 라벨 추출
labels = test['label'].values
labels

array([0, 0, 1, ..., 1, 1, 1])

In [26]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 비싸게 구매했는데 귀가 너무 불편해요. 아파요ㅜㅜ [SEP]
['[CLS]', '비', '##싸', '##게', '구', '##매', '##했', '##는데', '귀', '##가', '너', '##무', '불', '##편', '##해', '##요', '.', '[UNK]', '[SEP]']


In [27]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9379, 119089,  14153,   8908, 100372, 119424,  41850,
         8920,  11287,   9004,  32537,   9368,  50450,  14523,  48549,
          119,    100,    102,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [28]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [29]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([   101,   9379, 119089,  14153,   8908, 100372, 119424,  41850,   8920,
         11287,   9004,  32537,   9368,  50450,  14523,  48549,    119,    100,
           102,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [30]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

### **모델 생성**

In [31]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [32]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [33]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [34]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 2

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader)/2 * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

### **학습**

In [35]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [36]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [37]:
# 재현을 위해 랜덤시드 고정
seed_val = 15
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 2 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:05:42.
  Batch 1,000  of  4,219.    Elapsed: 0:11:35.
  Batch 1,500  of  4,219.    Elapsed: 0:17:29.
  Batch 2,000  of  4,219.    Elapsed: 0:23:23.
  Batch 2,500  of  4,219.    Elapsed: 0:29:17.
  Batch 3,000  of  4,219.    Elapsed: 0:35:12.
  Batch 3,500  of  4,219.    Elapsed: 0:41:05.
  Batch 4,000  of  4,219.    Elapsed: 0:46:58.

  Average training loss: 0.26
  Training epcoh took: 0:49:33

Running Validation...
  Accuracy: 0.92
  Validation took: 0:01:52

======== Epoch 2 / 2 ========
Training...
  Batch   500  of  4,219.    Elapsed: 0:05:54.
  Batch 1,000  of  4,219.    Elapsed: 0:11:47.
  Batch 1,500  of  4,219.    Elapsed: 0:17:40.
  Batch 2,000  of  4,219.    Elapsed: 0:23:34.
  Batch 2,500  of  4,219.    Elapsed: 0:29:27.
  Batch 3,000  of  4,219.    Elapsed: 0:35:21.
  Batch 3,500  of  4,219.    Elapsed: 0:41:14.
  Batch 4,000  of  4,219.    Elapsed: 0:47:08.

  Average training loss: 0.20
  

### **테스트 dataset 평가**

In [38]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of  1,563.    Elapsed: 0:00:24.
  Batch   200  of  1,563.    Elapsed: 0:00:48.
  Batch   300  of  1,563.    Elapsed: 0:01:12.
  Batch   400  of  1,563.    Elapsed: 0:01:35.
  Batch   500  of  1,563.    Elapsed: 0:01:59.
  Batch   600  of  1,563.    Elapsed: 0:02:23.
  Batch   700  of  1,563.    Elapsed: 0:02:47.
  Batch   800  of  1,563.    Elapsed: 0:03:11.
  Batch   900  of  1,563.    Elapsed: 0:03:35.
  Batch 1,000  of  1,563.    Elapsed: 0:03:59.
  Batch 1,100  of  1,563.    Elapsed: 0:04:22.
  Batch 1,200  of  1,563.    Elapsed: 0:04:46.
  Batch 1,300  of  1,563.    Elapsed: 0:05:10.
  Batch 1,400  of  1,563.    Elapsed: 0:05:34.
  Batch 1,500  of  1,563.    Elapsed: 0:05:58.

Accuracy: 0.92
Test took: 0:06:13


### **새 데이터 테스트**

In [41]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [42]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [122]:
test_samples = '아이폰 쓰는 친구말론 아이폰 부심인지 그것보담은 별루지만 괜찮다는 반응그런데 문제는 음악들을때 외부소리 굳이 안켜도 너무 생생히 들리네요 음악소리 찢아져라 높이면 모를까 보통의 음량에선 버스방송소리 차 달리는 소리 너무 잘들려요 예전 브릿츠꺼는 첨엔 잘들려서 귀에 끼울때 한바퀴 잘돌려 귀에 꼭 맞추면 외부소리 음악안들어도 웅웅 거릴정도로만 들리는데 이건 음악을 들어도 생생하게 들려버려서 좋은건지 나쁜건지 외부소음 켜가는 사실상 안들리는 소리까지 섞여 들려버려서 오히려 무슨소리인지 알수 없어요 시끄러운 곳에서 조용히 음악듣기엔 실패인듯 오히려 걸으면서 듣기에 좋을듯하네요'

logits = test_sentences([test_samples])

print(logits)
print(np.argmax(logits))

[[ 0.54898936 -0.08062357]]
0


In [161]:
import pandas as pd
import plotly.express as px
from google.colab import files
files.upload()

{}

In [162]:
samsung = pd.read_csv("samsung.csv")
samsung.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  200 non-null    int64  
 1   ratings     200 non-null    int64  
 2   reviews     200 non-null    object 
 3   label       200 non-null    int64  
 4   label_t     0 non-null      float64
dtypes: float64(1), int64(3), object(1)
memory usage: 7.9+ KB


In [163]:
i=0
display(test_samples)

while i < 200 :
  logits = test_sentences([samsung.reviews[i]])
  samsung.label_t[i] = np.argmax(logits)
  i = i+1



'아이폰 쓰는 친구말론 아이폰 부심인지 그것보담은 별루지만 괜찮다는 반응그런데 문제는 음악들을때 외부소리 굳이 안켜도 너무 생생히 들리네요 음악소리 찢아져라 높이면 모를까 보통의 음량에선 버스방송소리 차 달리는 소리 너무 잘들려요 예전 브릿츠꺼는 첨엔 잘들려서 귀에 끼울때 한바퀴 잘돌려 귀에 꼭 맞추면 외부소리 음악안들어도 웅웅 거릴정도로만 들리는데 이건 음악을 들어도 생생하게 들려버려서 좋은건지 나쁜건지 외부소음 켜가는 사실상 안들리는 소리까지 섞여 들려버려서 오히려 무슨소리인지 알수 없어요 시끄러운 곳에서 조용히 음악듣기엔 실패인듯 오히려 걸으면서 듣기에 좋을듯하네요'

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [164]:
samsung['label_t'] = samsung['label_t'].astype('int')

In [165]:
samsung

,Unnamed: 0,ratings,reviews,label,label_t
0,0,5,사용시간이 상당히 길어진 느낌입니다이어폰 본체와 충전케이스 모두 배터리 용량이 증가...,1,1
1,1,5,배송이 진짜 빠르고 상품도 안전하게 잘 왔어요 주변 소리도 인지가 가능하고 차가 오...,1,1
2,2,5,내돈내산 내 인생 첫번째 블루투스 이어폰 취준생기간이 길어서 블루투스 이어폰은 꿈...,1,0
3,3,4,아빠 생신 선물로 샀어요 블루투스 이어폰 필요하다면서 제거 버즈 탐내길래 플러스로 ...,1,1
4,4,5,배송도 빠르고 제조연월도 20년 8월이더라구요처음에 열었을 때 탭에 바로 연결됐는데...,1,1
...,...,...,...,...,...
195,195,2,한달사용후기입니다구매한 이유 동생이 일 특성상 핸드폰을 들고 통화하기 어려워 사줬...,0,0
196,196,5,일요일에 주문했는데 월요일배송 화요일 오후에 도착했네요 오래걸릴까봐 걱정했는데 매우...,1,1
197,197,4,분홍색 판매하는 곳이 별로 없어서 어디서 살까 고민하다가 여기서 사게 되었는데 배송...,1,1
198,198,5,한달 동안 꾸준히 와이프가 사용했습니다에러 등 문제는 한번도 없었고 음질 기능 디자...,1,1


In [173]:
is_right = samsung[samsung['label']!=samsung['label_t']]
is_right

,Unnamed: 0,ratings,reviews,label,label_t
2,2,5,내돈내산 내 인생 첫번째 블루투스 이어폰 취준생기간이 길어서 블루투스 이어폰은 꿈...,1,0
5,5,5,유선블투 이어폰이 문득 불편해질 때즈음 지인이 쓰는거보고 한번 테스트후 고민끝에 질...,1,0
10,10,5,제품은 잘 받았습니다참 많은 고민을 하고 결정했습니다선이 없으니 잘 잊어먹을거 같다...,1,0
11,11,5,블루투스 커널형 이어폰중에는 가장 좋지 않나 싶어요아무래도 이어폰을 청소해줘야된다거...,1,0
12,12,5,갤럭시 버즈 라이브를 구매했지만 도저히 아니다 싶어서중고로 판매하고 결국에는 버즈 ...,1,0
13,13,4,블루 하려다 핑크 했는데 나쁘지 않습니다 하지만 다시 산다면 블루를 살거같아요 그리...,1,0
14,14,5,처음 받았을 때 상자 모퉁이가 찌그러져있어서 놀랐어요다행히 내용물은 이상이 없습니다...,1,0
15,15,5,9월 8일 주문했는데 10일에 도착했네요대체적으로 배송은 빠른것 같아요그리고 후기에...,1,0
18,18,5,주변 사람들이 왜 갤럭시버즈플러스를 추천하는지이제야 알겠네요이렇게 좋은건지 알았으면...,1,0
19,19,3,배송은 주문 다음 날 받았구요 9월 제조 제품입니다비싼 이어폰 쓰기전에 무선이어폰을...,1,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 58 entries, 2 to 199
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  58 non-null     int64 
 1   ratings     58 non-null     int64 
 2   reviews     58 non-null     object
 3   label       58 non-null     int64 
 4   label_t     58 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 2.7+ KB


In [187]:
is_right.to_csv("samsung_wrong.csv",encoding='euc-kr')

In [188]:
from google.colab import files
files.download("samsung_wrong.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [191]:
is_right.shape

(58, 5)

In [160]:
display(samsung['label']!=samsung['label_t'])

0      False
1      False
2       True
3      False
4      False
       ...  
195    False
196    False
197    False
198    False
199     True
Length: 200, dtype: bool

In [201]:
import sklearn

In [207]:
sklearn.metrics.confusion_matrix(samsung['label'], samsung['label_t'])

array([[  8,   0],
       [ 58, 134]])

In [211]:
metrics.f1_score(samsung['label'], samsung['label_t'])

0.8220858895705522

In [210]:

metrics.roc_auc_score(samsung['label'], samsung['label_t'])

0.8489583333333333

In [ ]:
F.softmax(logits)의 값과 labels의 값